# using gattlib

In [ ]:
import sys
from gattlib import GATTRequester
st_addres = "D5:C9:57:C2:25:7B"

In [ ]:
requester = GATTRequester(st_addres, False)
requester.connect(True, channel_type="random")

In [ ]:
requester.discover_primary()

In [ ]:
characteristics = requester.discover_characteristics()
characteristics

![alt text](Properties.png "Title")

In [ ]:
def gen(x):
    while x:
        yield bin(x % 16)[2:].zfill(4)
        x >>= 4

def nibble(x):
    return '0b'+'_'.join(list(gen(x))[::-1])

In [ ]:
nibble(0x00140000)

In [ ]:
nibble(0x00e00000)

In [ ]:
0x0010

In [ ]:
def print_handle_14(d):
    print('d:', [hex(di) for di in d])
    tick = int.from_bytes(d[:2], "little")
    print("tick:",tick)
    pressure = float(int.from_bytes(d[2:6], "little"))/100
    print("pressure:",pressure)
    temprature = float(int.from_bytes(d[6:], "little"))/10.0
    print("temprature:",temprature)

In [ ]:
# read data diraectly
data = requester.read_by_handle(0x000e)
print_handle_14(data[0])

In [ ]:
# start notification
requester.write_by_handle(15, b"\x01\x00")

In [ ]:
# stop notification
requester.write_by_handle(15, b"\x00\x00")

In [ ]:
requester.disconnect()
del requester

## using notifications to read data

In [ ]:
# read using notification on handle 0x0e (or 14)
from threading import Event


class Requester(GATTRequester):
    def __init__(self, wakeup, *args):
        GATTRequester.__init__(self, *args)
        self.wakeup = wakeup
        self.counter = 0

    def on_notification(self, handle, data):
        print("-------------------------------------------")
        print("- notification on handle: {}".format(handle))
        print(data)
        print_handle_14(data[-8:])
        # wait for 10 notifications
        self.counter = self.counter + 1
        if self.counter > 10:
            self.wakeup.set()


class ReceiveNotification(object):
    def __init__(self, address):
        self.received = Event()
        self.requester = Requester(self.received, address, False)
        self.connect()
    
    def main(self):
        self.wait_notification()

    def connect(self):
        print("Connecting...", end=' ')
        sys.stdout.flush()

        self.requester.connect(True, channel_type="random")
        print("OK!")

    def wait_notification(self):
        self.requester.write_by_handle(14+1, b"\x01\x00")
        print("\nThis is a bit tricky. You need to make your device to send\n"
              "some notification. I'll wait...")
        self.received.wait()
        # turn off notification
        self.requester.write_by_handle(14+1, b"\x00\x00")

p = ReceiveNotification(st_addres)
p.main()

In [ ]:
# p.requester.write_by_handle(14 + 1, b"\x00\x00")
p.requester.disconnect()

# using bluepy

In [ ]:
from bluepy import btle
st_addres = "D5:C9:57:C2:25:7B"

class MyDelegate(btle.DefaultDelegate):
    def __init__(self):
        btle.DefaultDelegate.__init__(self)

    def handleNotification(self, cHandle, data):
        print("A notification was received: {}".format(data))


p = btle.Peripheral(st_addres, btle.ADDR_TYPE_RANDOM)
p.setDelegate( MyDelegate() )

In [ ]:
# Setup to turn notifications on, e.g.
svc = p.getServiceByUUID('00000000-0002-11e1-9ab4-0002a5d5c51b')
ch = svc.getCharacteristics()[0]

# enable notification
p.writeCharacteristic(ch.valHandle+1, b"\x01\x00")

counter = 0

while True:
    if p.waitForNotifications(1.0):
        counter += 1
        # wait for five notification
        if counter > 5:
            break

# disable notification
p.writeCharacteristic(ch.valHandle+1, b"\x00\x00")

In [ ]:
char = p.getCharacteristics(uuid = '00e00000-0001-11e1-ac36-0002a5d5c51b')[0]
char.propertiesToString()
char.getHandle()

In [ ]:
srvs = p.getServices()
srvs = list(srvs)
[srv.uuid.getCommonName() for srv in srvs]

In [ ]:
chars = p.getCharacteristics()
chars = list(chars)
[srv.uuid.getCommonName() for srv in srvs]

In [ ]:
p.disconnect()

# using blue_st_sdk

In [ ]:
import sys
import os
import time
from abc import abstractmethod

from blue_st_sdk.manager import Manager
from blue_st_sdk.manager import ManagerListener
from blue_st_sdk.node import NodeListener
from blue_st_sdk.feature import FeatureListener
from blue_st_sdk.features.audio.adpcm.feature_audio_adpcm import FeatureAudioADPCM
from blue_st_sdk.features.audio.adpcm.feature_audio_adpcm_sync import FeatureAudioADPCMSync